<a href="https://colab.research.google.com/github/tayfununal/PyTorch/blob/main/to_Uniform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
import matplotlib.pyplot as plt


%matplotlib inline
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
from sklearn.datasets import load_digits


digits = load_digits()
X = digits.data
print('Data shape', X.shape)


def plot_digits(data, w=8):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(w, w),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))

plot_digits(digits.data)


In [ ]:
class Data(Dataset):
    def __init__(self, dataset):
        super(Data, self).__init__()

        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x = self.dataset[idx]
        return x

In [ ]:
class To_Uniform(torch.nn.Module):

    def __init__(self, input_dim, hidden_layer, output_dim):
        super(To_Uniform, self).__init__()

        self.input_dim = input_dim
        self.hidden_layer = hidden_layer
        self.output_dim = output_dim

        self.linear1 = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_layer),
            nn.ReLU(),
            nn.BatchNorm1d(self.hidden_layer)
        )
        self.linear2 = nn.Sequential(
            nn.Linear(self.hidden_layer, self.hidden_layer),
            nn.ReLU(),
            nn.BatchNorm1d(self.hidden_layer)
        )
        self.linear3 = nn.Sequential(
            nn.Linear(self.hidden_layer, self.hidden_layer),
            nn.ReLU(),
            nn.BatchNorm1d(self.hidden_layer)
        )
        self.linear4 = nn.Sequential(
            nn.Linear(self.hidden_layer, self.hidden_layer),
            nn.ReLU(),
            nn.BatchNorm1d(self.hidden_layer)
        )
        self.linear5 = nn.Sequential(
            nn.Linear(self.hidden_layer, self.output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.linear4(x)
        x = self.linear5(x)
        return x

    def criterion(self, y, y_hat):
      return - nn.MSELoss()(y, y_hat)

In [ ]:
X = digits.data
X = X.reshape(1797, 64)
X = (X - X.min()) / (X.max()-X.min())

X = Data(torch.from_numpy(X).type(torch.float32))
y = torch.from_numpy(digits.target).type(torch.float32)

In [ ]:
model = To_Uniform(64, 256, 24)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
train_loader = DataLoader(X, batch_size=1797, shuffle=True)

In [ ]:
epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

COST = []

for epoch in range(epochs):
    total_loss = []
    x_ = x.type(torch.float32) + torch.normal(mean=0.0, std=0.000001, size=(x.shape[0], 64)).type(torch.float32)
    for x in train_loader:

        y_hat = model(x_)

        loss = model.criterion(model(x), y_hat)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss.append(loss.item())

    COST.append(np.mean(total_loss))

    if epoch==0:
        print("Epoch : ", epoch+1, "  -----------> Loss : ", np.mean(total_loss))
    elif (epoch+1)%1 == 0:
        print("Epoch : ", epoch+1, "  -----------> Loss : ", np.mean(total_loss))

In [ ]:
Y = model(x)

In [ ]:
plt.scatter(Y[:,0].detach().numpy(), Y[:,1].detach().numpy(), c=y)
plt.show()